In [1]:
# 上下文管理器和else块

# else
# 不仅可以在if语句中使用，还可以在for、while和try语句中使用

In [2]:
# 上下文管理器类
class LookingGlass:
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero')
            return True

In [4]:
with LookingGlass() as what:
    print('aka.bac')
    print(what)

cab.aka
YKCOWREBBAJ


In [5]:
what

'JABBERWOCKY'

In [6]:
print('back to normal')

back to normal


In [1]:
# @contextmanager 
# 该装饰器能减少创建上下文管理器的样板代码量
# 因为不用编写一个完整的类，定义__enter__和__exit__方法
# 而只需要实现一个yield语句的生成器，生成想让__enter__方法返回的值

import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [2]:
with looking_glass() as what:
    print('aka.bac')
    print(what)

cab.aka
YKCOWREBBAJ


In [3]:
what

'JABBERWOCKY'

In [4]:
# 基于生成器的上下文管理器
# 而且实现了异常处理

import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)


In [9]:
# 协程 - 一个简单的演示
def simple_coroutine():
    print('-> corotine started')
    x = yield
    print('-> coroutine received:', x)

my_coro = simple_coroutine()
my_coro


<generator object simple_coroutine at 0x000001BA1DFF6F20>

In [10]:
next(my_coro)

-> corotine started


In [11]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [12]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [13]:
coro_avg = averager()
next(coro_avg)

In [14]:
coro_avg.send(10)

10.0

In [15]:
coro_avg.send(30)

20.0

In [17]:
coro_avg.send(5)

15.0

In [18]:
# 预激协程的装饰器
from functools import wraps

def coroutine(func):
    """装饰器：向前执行到第一个`yield`表达式，预激`func`"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer